In [1]:
import os

In [5]:
os.getcwd()

'/Volumes/JK/Swizzl/swizzl/services'

In [68]:
from bs4 import BeautifulSoup
import requests 
from mlearning import sentiment as st
from mlearning import prof
import re
import html

/Users/jagankaartik58/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/Users/jagankaartik58/anaconda3/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.20.2 when using version 0.21.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/jagankaartik58/anaconda3/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LinearSVC from version 0.20.2 when using version 0.21.0. This might lead to breaking code or invalid results. Use at your own ri

In [69]:
from bs4 import BeautifulSoup as BSHTML
import urllib.request

In [103]:
def linkImg(link):
    url = "{}".format(link)
    r = requests.get(url) 
    soup = BeautifulSoup(r.content,"lxml") 
    obj = soup.find_all("img",class_="Maw(100%)")
    for i in obj:
        return i['src']

In [86]:
def linkText(link):
    url = "{}".format(link)
    r = requests.get(url) 
    soup = BeautifulSoup(r.content,"lxml") 
    article_text = ''
    mylink = soup.find('div').findAll('p')
    for i in mylink:
        article_text += ''.join(i.findAll(text = True))
    return article_text

In [105]:
def YahooFetch():
    url = "https://news.yahoo.com/rss/"
    url = requests.get(url)
    soup = BeautifulSoup(url.text,'xml')
    
    try:
        # Fetch All Required items
        titles = soup.findAll('title')
        links = soup.findAll('link')
        pubDates = soup.findAll('pubDate')

    except Exception as e :
        
        # Return Empty if titles, links, descriptions, pubdates not found
        print(e)
        return "Error"

    # We don't want the first elements as these are just metadata
    
    titles.pop(0)
    titles.pop(0)
    links.pop(0)
    links.pop(0)
    pubDates.pop(0)

    #Dictionary to hold crawled information

    FeedDict = {}
    temp = []
    id = []
    
    try:
        # Add titles to the Dictionary
            
        
        for i in titles:
            temp.append(html.unescape(i.get_text()))
            FeedDict['title'] = temp
            
        # Add link and details regarding text contetn @ link to the Dictionary
        
        temp = []
        temptext = []
        tbscore = []
        vadscore = []
        profvalue = []
        for i in links:
            
            # Append Links
            temp.append(html.unescape(i.get_text()))
            
            # Append Text Content from Links
            textval = linkText(html.unescape(i.get_text()))
            textval = re.sub('\"','\\"',textval)
            textval = " \" " + textval + " \" "
            
            #Run LinkImg Finder
            temptext.append(linkImg(html.unescape(i.get_text())))
            
            # Find Subjectivity, Objectivity of text content
            score = st.sentimentTB(textval)
            tbscore.append(score)
            
            # Find Sentiment of text content
            score = st.sentimentVader(textval)
            vadscore.append(score)
            
            # Find Profanity Score of content
            textval = [textval]
            score = float(prof.predProf(textval))
            profvalue.append(score)
            
            # Add to Feeds Dictionary 
            FeedDict['link'] = temp
            FeedDict['linktext'] = temptext
            FeedDict['tbScore'] = tbscore
            FeedDict['vaderScore'] = vadscore
            FeedDict['prof'] = profvalue
            
        # Add Published Dates to the Dictionary
        temp = []
        for i in pubDates:
            temp.append(i.get_text())
            FeedDict['pubdate'] = temp
        
        print("Success")
    except Exception as e :
        print(e)
        return "Error"
    
    return FeedDict

In [106]:
FeedDict = YahooFetch()

Success


In [107]:
FeedDict

{'title': ['U.S. panel sets deadline for Trump to decide participation in impeachment hearings',
  '50,000 people allowed back home after chemical plant blasts in Texas',
  'Inmate wanted by ICE released on bail. He was arrested weeks later for attempted murder',
  "This Is America's Role in Saudi Arabia's Power Struggle",
  'Italy’s ‘Miss Hitler’ Among 19 Investigated for Starting New Nazi Party in Italy',
  'U.S. Rebukes Zambia for Jailing Two Men for Homosexuality',
  "Kamala Harris' campaign manager is under fire, receives blame for decline",
  'Families of Mexico massacre victims face backlash after cartel shooting',
  'Who made the new drapes? It’s among high court’s mysteries',
  "Pakistani man aims to bring shade to Iraq's Arbaeen pilgrims",
  '7 Homes for Sale in the Most Secluded Parts of the World',
  '2 victims were killed and police fatally shot a man wearing a hoax explosive vest in a terrorist attack at London Bridge',
  'NTSB: Nine people killed, three injured in South 